In [1]:
# https://github.com/itzmestar/duneanalytics
# Source: https://github.com/dansal0807/top-nfts-data-analysis/blob/main/Top_NFTs_Coleta%2C_Limpeza_e_Explora%C3%A7%C3%A3o_dos_dados.ipynb

In [2]:
# Importing the libraries
import pandas as pd
import requests
from requests_html import HTMLSession
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import seaborn as sns
import json
from duneanalytics import DuneAnalytics

In [3]:
# request nft collections data
session = HTMLSession()
r = session.get('https://www.coingecko.com/en/nft').text
df = pd.read_html(r)
nft_collection_data = df[0]
nft_collection_data

,#,NFT,Floor Price,24h,7d,Market Cap,24h Volume,Owners,24h Owners,Total Assets
0,1,We Are All Going to Die,1.77 ETH,60.9%,NaN,"11,798.82 ETH","1,764.62 ETH",3156.0,3.7%,6666.0
1,2,goblintown.wtf,5.45 ETH,1.1%,NaN,"54,494.55 ETH",917.48 ETH,4608.0,0.1%,9999.0
2,3,Mutant Ape Yacht Club,19.2 ETH,8.8%,NaN,"372,748.8 ETH",728.94 ETH,13006.0,2.3%,19414.0
3,4,Bored Ape Yacht Club,90.0 ETH,5.6%,NaN,"900,000.0 ETH",424.68 ETH,6388.0,1.3%,10000.0
4,5,Otherdeed for Otherside,2.6 ETH,-1.4%,NaN,"260,000.0 ETH",417.9 ETH,34286.0,-0.0%,100000.0
...,...,...,...,...,...,...,...,...,...,...
95,96,Fancy Bears,0.4 ETH,2.0%,NaN,"3,155.2 ETH",6.5 ETH,3615.0,0.0%,7888.0
96,97,MAX PAIN AND FRENS BY XCOPY,0.63 ETH,2.0%,NaN,"4,705.47 ETH",6.13 ETH,4125.0,0.1%,7469.0
97,98,MURI by Haus,0.27 ETH,-10.8%,NaN,"2,700.0 ETH",6.0 ETH,6054.0,-0.0%,10000.0
98,99,Phanta Bear,0.38 ETH,-1.6%,NaN,"3,800.0 ETH",5.49 ETH,5125.0,-0.0%,10000.0


In [4]:
dune_username = 'blueberrytaurus'
dune_password = 'RPbb3@FwM!'

In [5]:
dune_username = 'blueberrytaurus'
dune_password = 'RPbb3@FwM!'
dune = DuneAnalytics(dune_username, dune_password)
dune.login()
dune.fetch_auth_token()
# query id for any query can be found from the url of the query:
# https://duneanalytics.com/queries/4494/8769 => 4494
result_id = dune.query_result_id(query_id=131031)
# fetch query result
data_dict = dune.query_result(result_id)

{'data': {'get_result_v2': {'job_id': None, 'result_id': '25f4a6be-5bc9-4ce2-8a6d-b31937c87342', 'error_id': '28ceb09a-c6a9-438e-a02f-87f286558be0', '__typename': 'get_result_v2_response'}}}
{'data': {'query_results': [{'id': '25f4a6be-5bc9-4ce2-8a6d-b31937c87342', 'job_id': '3cfd3d1d-836d-44e8-bc57-56a6460e6418', 'error': None, 'runtime': 317, 'generated_at': '2022-05-24T23:44:07.603712+00:00', 'columns': ['timespan', 'total_volume', 'average_price', 'min_price', 'max_price', 'total_sales', 'unique_sellers', 'unique_buyers'], '__typename': 'query_results'}], 'get_result_by_result_id': [{'data': {'average_price': 143.0, 'max_price': 143.0, 'min_price': 143.0, 'timespan': '2022-05-02T23:00:00+00:00', 'total_sales': 1, 'total_volume': 143.0, 'unique_buyers': 1, 'unique_sellers': 1}, '__typename': 'get_result_template'}, {'data': {'average_price': 125.0, 'max_price': 125.0, 'min_price': 125.0, 'timespan': '2022-05-01T18:00:00+00:00', 'total_sales': 1, 'total_volume': 125.0, 'unique_buyers

In [6]:
# Parse dune response to dataframe
data = json.dumps(data_dict)
json_data = json.loads(data)
norm_json_data = pd.json_normalize(json_data)
nested_dic = norm_json_data['data.get_result_by_result_id'][0]
flatten_dic = [{**x, **x.pop('data')} for x in nested_dic]
nft_collection_marketdata = pd.DataFrame(flatten_dic)

In [7]:
# Delete unnecesary columns
del nft_collection_marketdata[nft_collection_marketdata.columns[0]]
nft_collection_marketdata['buyers_ratio'] = nft_collection_marketdata['unique_buyers']/nft_collection_marketdata['unique_sellers']
nft_collection_marketdata

,average_price,max_price,min_price,timespan,total_sales,total_volume,unique_buyers,unique_sellers,buyers_ratio
0,143.000000,143.000000,143.000000,2022-05-02T23:00:00+00:00,1,143.000000,1,1,1.000000
1,125.000000,125.000000,125.000000,2022-05-01T18:00:00+00:00,1,125.000000,1,1,1.000000
2,150.000000,150.000000,150.000000,2022-04-23T13:00:00+00:00,1,150.000000,1,1,1.000000
3,122.388000,122.388000,122.388000,2022-04-05T16:00:00+00:00,1,122.388000,1,1,1.000000
4,99.416835,99.416835,99.416835,2022-03-23T22:00:00+00:00,1,99.416835,1,1,1.000000
...,...,...,...,...,...,...,...,...,...
4839,0.100000,0.150000,0.040000,2021-05-01T03:00:00+00:00,9,0.900000,8,6,1.333333
4840,0.121250,0.200000,0.100000,2021-05-01T02:00:00+00:00,8,0.970000,5,4,1.250000
4841,0.080000,0.080000,0.080000,2021-05-01T01:00:00+00:00,1,0.080000,1,1,1.000000
4842,0.085000,0.100000,0.080000,2021-05-01T00:00:00+00:00,4,0.340000,2,3,0.666667


In [8]:
nft_collection_marketdata.to_csv('./data/nft_collection_marketdata.csv', index=False)

In [9]:
# Get NFT sale history by token
dune = DuneAnalytics(dune_username, dune_password)
dune.login()
dune.fetch_auth_token()
result_id = dune.query_result_id(query_id=882347)
nft_sale_dict = dune.query_result(result_id)

{'data': {'get_result_v2': {'job_id': 'f0b59def-efa2-450e-983c-5a6086e44bd3', 'result_id': '750af363-034a-42a7-af24-0ba7df68838a', 'error_id': None, '__typename': 'get_result_v2_response'}}}


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
# Parse dune response to dataframe
data = json.dumps(nft_sale_dict)
json_data = json.loads(data)
norm_json_data = pd.json_normalize(json_data)
nested_dic = norm_json_data['data.get_result_by_result_id'][0]
flatten_dic = [{**x, **x.pop('data')} for x in nested_dic]
nft_sale_history = pd.DataFrame(flatten_dic)

In [11]:
del nft_sale_history[nft_sale_history.columns[0]]
nft_sale_history

,eth_fees,eth_sale price,time,token_id,usd_platform_fees,usd_sale_price
0,0.09975,3.99,2021-08-29T01:05:16+00:00,1122,326.733120,13069.3248
1,0.09950,3.98,2021-08-29T01:06:20+00:00,116,325.914240,13036.5696
2,0.09950,3.98,2021-08-29T01:06:52+00:00,114,325.914240,13036.5696
3,0.09225,3.69,2021-08-29T01:06:52+00:00,771,302.166720,12086.6688
4,0.16650,6.66,2021-08-29T01:08:52+00:00,122,545.374080,21814.9632
...,...,...,...,...,...,...
29480,0.50625,20.25,2022-06-08T04:03:15+00:00,17718,905.306625,36212.2650
29481,0.57500,23.00,2022-06-08T04:31:25+00:00,6680,1027.950500,41118.0200
29482,0.57200,22.88,2022-06-08T06:39:34+00:00,21638,1030.904160,41236.1664
29483,0.48500,19.40,2022-06-08T06:54:34+00:00,29356,NaN,NaN


In [12]:
nft_sale_history.to_csv('./data/nft_sale_history.csv', index=False)